In [1]:
import math 
import numpy as np
from scipy.stats import uniform, expon, norm
from numpy import random

from ipywidgets import *
import ipywidgets as widgets
from ipysheet import from_array, to_array, sheet, cell

In [2]:
#Оператор один 
A1 = widgets.IntSlider(min=0, max=100, value = 15)
B1 = widgets.IntSlider(min=0, max=100, value = 25)
display(A1)
display(B1)

IntSlider(value=15)

IntSlider(value=25)

In [3]:
#Оператор два
A2 = widgets.IntSlider(min=0, max=100, value = 30)
B2 = widgets.IntSlider(min=0, max=100, value = 50)
display(A2)
display(B2)

IntSlider(value=30)

IntSlider(value=50)

In [4]:
#Оператор три
A3 = widgets.IntSlider(min=0, max=100, value = 20)
B3 = widgets.IntSlider(min=0, max=100, value = 60)
display(A3)
display(B3)

IntSlider(value=20)

IntSlider(value=60)

In [5]:
#Клиенты
Cin = widgets.IntSlider(min=0, max=100, value = 8)
Cout = widgets.IntSlider(min=0, max=100, value = 12)
display(Cin)
display(Cout)

IntSlider(value=8)

IntSlider(value=12)

In [6]:
#Компьютер раз
C1i = widgets.FloatSlider(min=0.0, max=20.0, value = 15.0)
C1o = widgets.FloatSlider(min=0.0, max=20.0, value = 15.1)
display(C1i)
display(C1o)

FloatSlider(value=15.0, max=20.0)

FloatSlider(value=15.1, max=20.0)

In [7]:
#Компьютер два
C2i = widgets.FloatSlider(min=0.0, max=20.0, value = 30.0)
C2o = widgets.FloatSlider(min=0.0, max=20.0, value = 30.1)
display(C2i)
display(C2o)

FloatSlider(value=20.0, max=20.0)

FloatSlider(value=20.0, max=20.0)

In [8]:
#Шаг по времени
delta = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value = 0.01)
display(delta)

FloatSlider(value=0.01, max=1.0, step=0.01)

In [9]:
N = widgets.IntSlider(min=0, max=1000, value = 300)
display(N)

IntSlider(value=300, max=1000)

In [10]:
#генерация 
def generate_next(a, b):
    return random.uniform(a, b)

In [11]:
#оставшееся время
t_req = generate_next(Cin.value, Cout.value) #когда следующая заявка
t_o1 = math.inf
t_o2 = math.inf
t_o3 = math.inf

t_c1 = math.inf
t_c2 = math.inf

In [12]:
que_r = 0
que_c1 = 0
que_c2 = 0

In [13]:
mt = 0 #время моделирования
reqa = 0 #первый комп
reqb = 0 #второй комп
unreq = 0 #заявки необработанные 

In [14]:
#список событий
class myEvent:
    def __init__(self, name, time):
        self.name = name
        self.time = time
        
ev_req = myEvent("req", t_req)
ev_o1 = myEvent("o1", t_o1)
ev_o2 = myEvent("o2", t_o2)
ev_o3 = myEvent("o3", t_o3)

ev_c1 = myEvent("c1", t_c1)
ev_c2 = myEvent("c2", t_c2)

times = [ev_req, ev_o1, ev_o2, ev_o3, ev_c1, ev_c2] 

b_o1 = False
b_o2 = False
b_o3 = False

b_c1 = False
b_c2 = False

In [15]:
while reqa + reqb < N.value:
    mt += times[0].time
    for i in range(len(times) - 1, -1, -1):
        times[i].time -= times[0].time
        
    if times[0].name == 'o1':
        b_o1 = False
        ev_o1.time = math.inf
        if not b_c1:
            b_c1 = True
            ev_c1.time = generate_next(C1i.value, C1o.value)
        else:
            que_c1 = que_c1 + 1         
    elif times[0].name == 'o2':
        b_o2 = False
        ev_o2.time = math.inf
        if not b_c1:
            b_c1 = True
            ev_c1.time = generate_next(C1i.value, C1o.value)
        else:
            que_c1 = que_c1 + 1           
    elif times[0].name == 'o3':
        b_o3 = False
        ev_o3.time = math.inf
        if not b_c2:
            b_c2 = True
            ev_c2.time = generate_next(C2i.value, C2o.value)
        else:
            que_c2 += 1    
    elif times[0].name == 'c1':
        reqa += 1        
        b_c1 = False
        ev_c1.time = math.inf
        
        if que_c1 > 0:
            que_c1 -= 1
            b_c1 = True
            ev_c1.time = generate_next(C1i.value, C1o.value)
            
    elif times[0].name == 'c2':
        reqb += 1        
        b_c2 = False
        ev_c2.time = math.inf
        
        if que_c2 > 0:
            que_c2 -= 1
            b_c2 = True
            ev_c2.time = generate_next(C2i.value, C2o.value)
    else:
        if not b_o1:
            b_o1 = True
            ev_o1.time = generate_next(A1.value, B1.value)
        elif not b_o2:
            b_o2 = True
            ev_o2.time = generate_next(A2.value, B2.value)
        elif not b_o3:
            b_o3 = True
            ev_o3.time = generate_next(A3.value, B3.value)
        else:
            unreq += 1
        ev_req.time = generate_next(Cin.value, Cout.value)

    times = sorted(times, key = (lambda x: x.time) )
    
    

In [16]:
s1 = sheet(5, 1, column_headers = list(['Всего заявок', 'Обработано', 'Не обработано', 'Время моделирования', 'Вероятность отказа']))
cell(0, 0, reqa + reqb + unreq)
cell(0, 1, reqa + reqb) 
cell(0, 2, unreq)
cell(0, 3, mt)
cell(0, 4, unreq / (reqa + reqb + unreq))
s1

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='numeric', value=385), Cell(colum…